In [88]:
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from skimage.feature import hog, local_binary_pattern
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import hamming_loss
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier


In [89]:
# resize with keeping ratio
def resize_with_padding(img, target_size=128):
    h, w = img.shape[:2]
    scale = target_size / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(img, (new_w, new_h))
    result = np.zeros((target_size, target_size, 3), dtype=np.uint8)
    x_offset = (target_size - new_w) // 2
    y_offset = (target_size - new_h) // 2
    result[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized
    return result

In [90]:
def lbp_histogram(gray, P=8, R=1, method='uniform'):
    lbp = local_binary_pattern(gray, P=P, R=R, method=method)
    hist, _ = np.histogram(lbp.ravel(), bins=59, range=(0, 59))
    hist = hist.astype('float32')
    hist = hist / (hist.sum() + 1e-8)
    return hist

In [91]:
def color_hist(img, bins=16):
    hist_b = cv2.calcHist([img], [0], None, [bins], [0, 256])
    hist_g = cv2.calcHist([img], [1], None, [bins], [0, 256])
    hist_r = cv2.calcHist([img], [2], None, [bins], [0, 256])
    hist = np.concatenate([hist_b, hist_g, hist_r]).ravel()
    hist = hist / (hist.sum() + 1e-8)
    return hist

In [92]:
def extract_hog(gray):
    return hog(gray, orientations=9, pixels_per_cell=(8,8), cells_per_block=(2,2), block_norm='L2-Hys')

In [93]:
with open(f'data/train/_annotations.coco.json') as f:
    train = json.load(f)
print(train.keys())

dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])


In [94]:
train_images = train['images']
train_labels = pd.DataFrame(train['annotations'])
categories = train['categories']
len(train_images) , len(train_labels)

(2571, 4044)

In [95]:
categories

[{'id': 0, 'name': 'Recycling-Waste', 'supercategory': 'none'},
 {'id': 1, 'name': 'cardboard', 'supercategory': 'Recycling-Waste'},
 {'id': 2, 'name': 'glass', 'supercategory': 'Recycling-Waste'},
 {'id': 3, 'name': 'metal', 'supercategory': 'Recycling-Waste'},
 {'id': 4, 'name': 'paper', 'supercategory': 'Recycling-Waste'},
 {'id': 5, 'name': 'plastic', 'supercategory': 'Recycling-Waste'}]

In [96]:
train_images[:1]

[{'id': 0,
  'license': 1,
  'file_name': 'metal152_jpg.rf.06414376028f03cc3a360e50505abb0f.jpg',
  'height': 384,
  'width': 512,
  'date_captured': '2025-11-30T15:32:11+00:00',
  'extra': {'name': 'metal152.jpg'}}]

In [97]:
train_labels[:1]

,id,image_id,category_id,bbox,area,segmentation,iscrowd
0,1,0,3,"[172, 152, 114, 93]",10602,[],0


In [98]:
train_labels.groupby('image_id').size().sort_values()

image_id
50       1
51       1
52       1
53       1
54       1
        ..
573     11
532     11
1681    11
1693    11
2125    13
Length: 2571, dtype: int64

In [99]:
def make_x_y(part):
    with open(f'data/{part}/_annotations.coco.json') as f:
        data = json.load(f)

    images = data['images']
    labels = pd.DataFrame(data['annotations'])

    labels_per_image = labels.groupby("image_id")["category_id"].apply(lambda x: sorted(set(x)))

    mlb = MultiLabelBinarizer(classes=[0,1,2,3,4,5])
    binary_labels = mlb.fit_transform(labels_per_image)

    y_data = pd.DataFrame(binary_labels, index=labels_per_image.index, columns=mlb.classes_)
    
    X_hog = []
    X_lbp = []
    X_color_hist = []

    for image in images:
        img = cv2.imread(f'data/{part}/{image["file_name"]}')
        img = resize_with_padding(img)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        normal_gray = cv2.normalize(gray.astype("float32"), None, alpha=0, beta=1,
                            norm_type=cv2.NORM_MINMAX)
        X_hog.append(extract_hog(normal_gray))
        X_lbp.append(lbp_histogram(gray))
        X_color_hist.append(color_hist(img))

    X_hog = np.array(X_hog)
    X_lbp = np.array(X_lbp)
    X_color_hist = np.array(X_color_hist)

    return X_hog, X_lbp, X_color_hist, y_data

In [100]:
X_hog_train,X_lbp_train, X_color_train, y_train = make_x_y('train')
X_hog_valid,X_lbp_valid, X_color_valid, y_valid= make_x_y('valid')
X_hog_test,X_lbp_test, X_color_test, y_test = make_x_y('test')

In [101]:
X_hog_train.shape, y_train.shape, X_hog_valid.shape, y_valid.shape

((2571, 8100), (2571, 6), (973, 8100), (973, 6))

In [102]:
X_hog = np.vstack([X_hog_train, X_hog_valid])
X_lbp = np.vstack([X_lbp_train, X_lbp_valid])
X_color = np.vstack([X_color_train, X_color_valid])
y = pd.concat([y_train, y_valid], axis=0).reset_index(drop=True)

In [103]:
X_hog.shape, y.shape

((3544, 8100), (3544, 6))

In [104]:
pca = PCA(n_components=300, random_state=7)   
X_hog_pca = pca.fit_transform(X_hog)
X_hog_test_pca = pca.transform(X_hog_test)


In [105]:
X = np.hstack([X_hog_pca, X_lbp, X_color])
X_test  = np.hstack([X_hog_test_pca,  X_lbp_test,  X_color_test])

In [106]:
base_svc = SVC()
multi_clf = MultiOutputClassifier(base_svc)

param_grid = {
    "estimator__kernel": ["rbf", "poly", "sigmoid", "linear"],
    "estimator__C": [0.1, 1, 10],
    "estimator__gamma": ["scale", "auto", 0.01],
    "estimator__degree": [2, 3],
}

grid = GridSearchCV(
    estimator=multi_clf,
    param_grid=param_grid,
    scoring="f1_micro",
    cv=5,
    n_jobs=-1
)

grid.fit(X, y)  
y_pred = grid.predict(X_test)


ValueError: 
All the 360 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
360 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\davood\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\davood\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\multioutput.py", line 547, in fit
    super().fit(X, Y, sample_weight=sample_weight, **fit_params)
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\davood\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_met

In [ ]:
hamming_loss(y_test, y_pred)